# Predicting future energy usage from multiple dependent time series

For updates on the way Sagemaker or AWS behave compared to the notebook code, please refer to https://livebook.manning.com/#!/book/machine-learning-for-business/chapter-6/v-5/102

## Part 1: Load and examine the data

In [4]:
data_bucket = 'martin-mlforbusiness-eu'
subfolder = 'ch06'
dataset = '06_meter_data.csv'

In [ ]:
pip install boto3==1.26.34

In [2]:
%matplotlib inline

from dateutil.parser import parse
import json
from random import shuffle
import random
import datetime
import os
                            
import boto3
import s3fs
import sagemaker
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

role = sagemaker.get_execution_role()
s3 = s3fs.S3FileSystem(anon=False)

In [5]:
s3_data_path = f"s3://{data_bucket}/{subfolder}/data"
s3_output_path = f"s3://{data_bucket}/{subfolder}/output"
df = pd.read_csv(f's3://{data_bucket}/{subfolder}/{dataset}', index_col=0)
df.head()


,Site_1,Site_2,Site_3,Site_4,Site_5,Site_6,Site_7,Site_8,Site_9,Site_10,...,Site_39,Site_40,Site_41,Site_42,Site_43,Site_44,Site_45,Site_47,Site_46,Site_48
2017-11-01 00:00:00,13.30,13.3,11.68,13.02,0.0,102.9,11.4,21.8,9.7,11.9,...,22.20,0.0,0.0,0,23.30,36.74,48.48,68.70,32.13,19.39
2017-11-01 00:30:00,11.75,11.9,12.63,13.36,0.0,122.1,11.3,17.7,9.0,12.4,...,21.68,0.0,0.0,0,22.14,36.44,50.18,69.09,29.02,19.54
2017-11-01 01:00:00,12.58,11.4,11.86,13.04,0.0,110.3,10.9,17.5,9.1,12.0,...,21.56,0.0,0.0,0,21.88,35.93,49.44,67.52,26.65,20.10
2017-11-01 01:30:00,12.50,10.8,11.53,11.83,0.0,83.6,11.2,16.5,12.4,11.5,...,21.28,0.0,0.0,0,22.69,45.25,49.57,68.48,25.28,19.46
2017-11-01 02:00:00,12.98,12.0,11.57,12.25,0.0,91.4,10.9,16.3,12.4,11.3,...,40.48,0.0,0.0,0,22.75,62.94,48.58,75.30,23.65,19.18


Erläuterung oben: Für 13 Monate wurde alle 30 Minuten der aktuelle Energiebedarf für alle 48 Standorte (Spalten) ermittelt.

In [6]:
print(f'Number of rows in dataset: {df.shape[0]}')
print(f'Number of columns in dataset: {df.shape[1]}')

Number of rows in dataset: 19632
Number of columns in dataset: 48


## Part 2 : Get the data in the right shape

Zwar bieten uns die Datenpunkte alle 30 Minuten eine sehr gute Basis um etwaige Peaks zu identifizieren, jedoch ist dieses Intervall für unsere Aufgabe zu feingranular. Für die weitere Verarbeitung der Daten wollen wir daher den Energiebedarf auf die Einheit "Tag" summieren.
Dafür wird zunächst der Index des Dataframes in ein datetime-Format umgewandelt. Anschließend sind wir in der Lage durch die resample-Funktion anhand des Tages ('D‘) zu summieren.

In [ ]:
df.index = pd.to_datetime(df.index)
daily_df = df.resample('D').sum()
daily_df.head()

In [ ]:
print(daily_df.shape)
print(f'Time series starts at {daily_df.index[0]} and ends at {daily_df.index[-1]}')

## Frage 1

**Wie sind die beiden Werte (409, 48) zu interpretieren?**

**Handling fehlender Werte**

fillna() ersetzt fehlende Werte mit dem vorherigen Wert. 
Da wir jedoch wissen, dass der Wert der Vorwoche ggb dem Vortag eine bessere Aussagekraft hat, setzen wir shift(7). So ersetzten wir ggf. einen fehlenden Montag-Wert mit dem Montag-Wert der Vorwoche, statt mit dem Sonntag-Wert.

In [ ]:
daily_df = daily_df.fillna(daily_df.shift(7))
daily_df

Anschließend wollen wir uns die Time-Series-Daten wieder visualisieren, um ein Gefühl für die Daten zu bekommen.

In [ ]:
print('Number of time series:',daily_df.shape[1])
fig, axs = plt.subplots(5, 2, figsize=(20, 20), sharex=True)
axx = axs.ravel()
indices = [0,1,2,3,4,5,40,41,42,43]
for i in indices:
    plot_num = indices.index(i)
    daily_df[daily_df.columns[i]].loc["2017-11-01":"2018-01-31"].plot(ax=axx[plot_num])
    axx[plot_num].set_xlabel("date")    
    axx[plot_num].set_ylabel("kW consumption")

Optisch gibt es einige auffällige Korrelationen, die DeepAR wahrscheinlich erkennen und nutzen wird!

## Part 3: Erstellen des Trainings- und Test-Datasets

DeepAR erwartet den Input (also die Zeitreihen) in JSON-Dateien. Um die JSON-Datei zu erstellen, müssen wir die Daten ein wenig bearbeiten:
>1. Konvertierung der Daten von einem DataFrame in eine Liste von Reihen
>2. Zurückhalten von 30 Tagen aus dem Trainingsdatensatz, damit das Modell nicht mit Daten trainiert wird, die es zum Testen verwenden soll
>3. Erstellen der JSON-Dateien

In [ ]:
daily_power_consumption_per_site = []
for column in daily_df.columns:
    site_consumption = np.trim_zeros(daily_df[column], trim='f')
    site_consumption = site_consumption.fillna(0)
    daily_power_consumption_per_site.append(site_consumption)
    
print(f'Time series covers {len(daily_power_consumption_per_site[0])} days.')
print(f'Time series starts at {daily_power_consumption_per_site[0].index[0]}')
print(f'Time series ends at {daily_power_consumption_per_site[0].index[-1]}') 

Wir legen fest, welche Tage für das Trainings- bzw. für das Testdatenset als Begrezung gelten sollen.

In [ ]:
freq = 'D' #frequenz: Tage (days)
prediction_length = 30 #Tage die unser Modell vorhersagen soll, 
#diese müssen wir als test-datenset von der Zeitreihe "abschneiden"
#anmerkung: die fehlenden 15 tage sind dezember

start_date = pd.Timestamp("2017-11-01 00:00:00", freq=freq)
end_training = start_date + 364 * start_date.freq
end_testing = end_training + prediction_length * start_date.freq

print(f'End training: {end_training}, End testing: {end_testing}')

Nun erstellen wir eine python-Liste. Diese wird im nächsten Schritt in JSON-Dateien gespeichert, die DeepAR verarbeiten kann. Dabei werden zwei Variablen übergeben "start" und "target". Also legen wir für "training" und "test" jeweils eine Liste an, die anschließend in ein json-File überführt und im s3-Bucket abgelegt wird.

In [ ]:
training_data = [
    {
        "start": str(start_date),
        "target": ts[start_date:end_training].tolist()
    }
    for ts in daily_power_consumption_per_site
]

test_data = [
    {
        "start": str(start_date),
        "target": ts[start_date:end_testing].tolist()
    }
    for ts in daily_power_consumption_per_site
]

In [ ]:
def write_dicts_to_s3(path, data):
    with s3.open(path, 'wb') as f:
        for d in data:
            f.write(json.dumps(d).encode("utf-8"))
            f.write("\n".encode('utf-8'))
            
write_dicts_to_s3(f'{s3_data_path}/train/train.json', training_data)
write_dicts_to_s3(f'{s3_data_path}/test/test.json', test_data)

## Part 4: Train the Model

Anschließend wird wieder das Modell trainiert. Wir wählen das "forecasting-deepar"-model.

In [ ]:
s3_output_path = f's3://{data_bucket}/{subfolder}/output'
sess = sagemaker.Session()

from sagemaker import image_uris 
image_name = sagemaker.image_uris.retrieve("forecasting-deepar", sess.boto_region_name, "latest")

Wir richten den Estimator ein und übergeben die sagemaker-Einstellungen für das Training.

In [ ]:
estimator = sagemaker.estimator.Estimator(
    sagemaker_session=sess,
    image_uri=image_name,
    role=role,
    instance_count=1,
    instance_type='ml.m4.4xlarge',
    base_job_name='ch6-energy-usage',
    output_path=s3_output_path
)

## Hyperparamtertuning

Für das Hyperparametertuning stehen für uns in diesem Modell insbesondere context_length und prediction_length im Fokus:

**context_length**
The number of time-points that the model gets to see before making the prediction.

**prediction_length**
The number of time-steps that the model is trained to predict, also called the forecast horizon. The trained model always generates forecasts with this length. It can't generate longer forecasts. The prediction_length is fixed when a model is trained and it cannot be changed later.

**Weitere Hyperparamter**
Link: https://docs.aws.amazon.com/sagemaker/latest/dg/deepar_hyperparameters.html

In [ ]:
estimator.set_hyperparameters(
    time_freq=freq,
    epochs="400",
    early_stopping_patience="40",
    mini_batch_size="64",
    learning_rate="5E-4",
    context_length="90",
    prediction_length=str(prediction_length)
)

Nun findet das tatsächliche Training statt.

In [ ]:
%%time
data_channels = {
    "train": "{}/train/".format(s3_data_path),
    "test": "{}/test/".format(s3_data_path)
}
estimator.fit(inputs=data_channels, wait=True)

## Part 5: Host the model

In [ ]:
endpoint_name = 'energy-usage'

try:
    sess.delete_endpoint(endpoint_name)
    sess.delete_endpoint_config(endpoint_name)
    print('Warning: Existing endpoint and configuration deleted to make way for your new endpoint.')
except:
    pass

In [ ]:
from sagemaker.serializers import IdentitySerializer

In der folgenden Zelle wird die DeepARPredictor-Klasse definiert, die im folgenden nutzen wollen. Diesen sehr technischen Part müssen wir nur ausführen, aber nicht detailliert nachvollziehen.  

In [ ]:
class DeepARPredictor(sagemaker.predictor.Predictor):
    def __init__(self, *args, **kwargs):
        super().__init__(
            *args,
            # serializer=JSONSerializer(),
            serializer=IdentitySerializer(content_type="application/json"),
            **kwargs,
        )

    def predict(
        self,
        ts,
        cat=None,
        dynamic_feat=None,
        num_samples=100,
        return_samples=False,
        quantiles=["0.1", "0.5", "0.9"],
    ):
        """Requests the prediction of for the time series listed in `ts`, each with the (optional)
        corresponding category listed in `cat`.

        ts -- `pandas.Series` object, the time series to predict
        cat -- integer, the group associated to the time series (default: None)
        num_samples -- integer, number of samples to compute at prediction time (default: 100)
        return_samples -- boolean indicating whether to include samples in the response (default: False)
        quantiles -- list of strings specifying the quantiles to compute (default: ["0.1", "0.5", "0.9"])

        Return value: list of `pandas.DataFrame` objects, each containing the predictions
        """
        prediction_time = ts.index[-1] + ts.index.freq
        quantiles = [str(q) for q in quantiles]
        req = self.__encode_request(ts, cat, dynamic_feat, num_samples, return_samples, quantiles)
        res = super(DeepARPredictor, self).predict(req)
        return self.__decode_response(res, ts.index.freq, prediction_time, return_samples)

    def __encode_request(self, ts, cat, dynamic_feat, num_samples, return_samples, quantiles):
        instance = series_to_dict(
            ts, cat if cat is not None else None, dynamic_feat if dynamic_feat else None
        )

        configuration = {
            "num_samples": num_samples,
            "output_types": ["quantiles", "samples"] if return_samples else ["quantiles"],
            "quantiles": quantiles,
        }

        http_request_data = {"instances": [instance], "configuration": configuration}

        return json.dumps(http_request_data).encode("utf-8")

    def __decode_response(self, response, freq, prediction_time, return_samples):
        # we only sent one time series so we only receive one in return
        # however, if possible one will pass multiple time series as predictions will then be faster
        predictions = json.loads(response.decode("utf-8"))["predictions"][0]
        prediction_length = len(next(iter(predictions["quantiles"].values())))
        prediction_index = pd.date_range(
            start=prediction_time, freq=freq, periods=prediction_length
        )
        if return_samples:
            dict_of_samples = {"sample_" + str(i): s for i, s in enumerate(predictions["samples"])}
        else:
            dict_of_samples = {}
        return pd.DataFrame(
            data={**predictions["quantiles"], **dict_of_samples}, index=prediction_index
        )

    def set_frequency(self, freq):
        self.freq = freq


def encode_target(ts):
    return [x if np.isfinite(x) else "NaN" for x in ts]


def series_to_dict(ts, cat=None, dynamic_feat=None):
    """Given a pandas.Series object, returns a dictionary encoding the time series.

    ts -- a pands.Series object with the target time series
    cat -- an integer indicating the time series category

    Return value: a dictionary
    """
    obj = {"start": str(ts.index[0]), "target": encode_target(ts)}
    if cat is not None:
        obj["cat"] = cat
    if dynamic_feat is not None:
        obj["dynamic_feat"] = dynamic_feat
    return obj

Jetzt können wir das Modell bereitstellen und einen Endpunkt erstellen, der mit unserer benutzerdefinierten DeepARPredictor-Klasse abgefragt werden kann.

In [ ]:
%%time
predictor = estimator.deploy(
    initial_instance_count=1,
    instance_type='ml.m5.large',
    predictor_cls=DeepARPredictor,
    endpoint_name=endpoint_name
)

## Part 6: Make Predictions and Plot Results 

Das 0.1-Quantil entspricht hier der unteren Grenze eines 80%-Konfidenzintervalls während das 0.9-Quantil der oberen Grenze eines 80%-Konfidenzintervalls entspricht. 

In [ ]:
predictor.predict(
    ts=daily_power_consumption_per_site[0][start_date+30*start_date.freq:end_training],
    quantiles=[0.1, 0.5, 0.9]
    ).head()

Nun gilt es noch die Funktion anzulegen, die uns die Vorhersage plottet.

In [ ]:
def plot(
    predictor, 
    target_ts,
    end_training=end_training, 
    plot_weeks=12,
    confidence=80
):
    frq = end_training.freq
    print(f"Calling served model to generate predictions from {end_training} to {end_training+prediction_length*frq}")
    low_quantile = 0.5 - confidence * 0.005
    up_quantile = confidence * 0.005 + 0.5
        
    plot_history = plot_weeks * 7

    fig = plt.figure(figsize=(20, 3))
    ax = plt.subplot(1,1,1)
    
    #vorhersagen
    prediction = predictor.predict(ts=target_ts[:end_training], quantiles=[low_quantile, 0.5, up_quantile])
    
    #tatsächliche Testset-Werte
    target_section = target_ts[end_training-plot_history*frq:end_training+prediction_length*frq]
    target_section.plot(color="black", label='Actual')
    
    ax.fill_between(
        prediction[str(low_quantile)].index, 
        prediction[str(low_quantile)].values, 
        prediction[str(up_quantile)].values, 
        color="b", alpha=0.3, label=f'{confidence}% confidence interval'
    )
    
    prediction["0.5"].plot(color="red", label='P50')
    #P50 (0.5) - The true value is expected to be lower than the predicted value 50% of the time. 
    #This is also known as the median forecast
    
    ax.legend(loc=2)    
    
    ax.set_ylim(target_section.min() * 0.5, target_section.max() * 1.5)

## Plotten der Vorhersagen

Die Diagramme zeigen den vorhergesagten (rot) gegenüber dem tatsächlichen (schwarz) Energieverbrauch für die Standorte (*sites = [...]*) im November 2018. Auffällig dabei ist, dass die Diagramme durchaus unterschiedliche Ausprägungen haben. So ist sites[0] durch eher wenige Ausschläge geprägt, während sites[33] deutlich einen wöchentlichen Verlauf erkennen lässt. Trotzdem gibt das Modell zuverlässige Vorhersagen zu beiden Standorten.

*(Das Konfidenzintervall gibt den Bereich an, der mit einer gewissen Wahrscheinlichkeit den Parameter einer Verteilung einer Zufallsvariablen einschließt.)*


In [ ]:
sites = [0,1,26,33,39,42,47]
for i in sites:
    plot_num = sites.index(i)
    plot(
        predictor,
        target_ts=daily_power_consumption_per_site[i][start_date+30*start_date.freq:],
        plot_weeks=2,
        confidence=80
    )

## Frage 2

**Wie sieht die Vorhersage für die Standorte 6,7,8 mit einem Vorlauf von 12 Wochen aus, wenn wir ein Konfidenzintervall von 95% darstellen wollen?**

**Wie sind diese Ausgaben zu interpretieren?**

## Berechnung von objektiven Statistiken zur Genauigkeit unseres Modells mit dem Test-Datenset

**MAPE** misst den "Mean Absolute Percentage Error" (mittleren absoluten prozentualen Fehler). Der Hauptgrund für die Verwendung von MAPE gegenüber dem RMSE ("Root Mean Squared Error" aus dem Modelltraining) besteht darin, dass er die Fehler in Prozenten und nicht in absoluten Werten bewertet. Eine Vorhersage von 11 für einen Wert von 10 wird also genauso behandelt gleich behandelt wie eine Vorhersage von 90 für einen Wert von 100.

Link: https://docs.aws.amazon.com/forecast/latest/dg/metrics.html#metrics-mape

In [ ]:
# Sammeln von 30-Tage-Vorhersagen über alle Zeitreihen
predictions= []
for i, ts in enumerate(daily_power_consumption_per_site):
    # Aufruf der prediction
    predictions.append(predictor.predict(ts=ts[start_date+30*start_date.freq:end_training])['0.5'].sum())

usages = [ts[end_training+1*start_date.freq:end_training+30*start_date.freq].sum() \
          for ts in daily_power_consumption_per_site]

for p,u in zip(predictions,usages):
    print(f'Predicted {p} kwh but usage was {u} kwh,')

In [ ]:
def mape(y_true, y_pred): 
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100

In [ ]:
print(f'MAPE: {round(mape(usages, predictions),1)}%')

## Frage 3

**Warum ist für diesen Anwendungsfall der MAPE eher geeignet als der RMSE?**

## Remove the Endpoint (recommended)

Comment out this cell to remove the endpoint if you want the endpoint to exist after "run all"

In [ ]:
sess.delete_endpoint(endpoint_name)
sess.delete_endpoint_config(endpoint_name)